In [ ]:
import torch
import time
import datasets
from transformers import AutoTokenizer, AutoModelForCausalLM
from trl import DataCollatorForCompletionOnlyLM
from utils import evaluate_model
import config

In [2]:
tokenizer = AutoTokenizer.from_pretrained(config.student_model_name)
response_template_ids = tokenizer("<|im_start|>assistant\n")["input_ids"]
collator = DataCollatorForCompletionOnlyLM(response_template_ids, tokenizer=tokenizer)

In [3]:
dataset = datasets.load_from_disk("/scratch/klambert/dataset/tulu-3-sft-mixture-pretokenized")
logit_dataset = datasets.load_from_disk("/scratch/klambert/slm_ensembles/teacher_logits/teacher_logits")

In [4]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['chat_text', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 192698
    })
    test: Dataset({
        features: ['chat_text', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1949
    })
})


In [5]:
print(torch.cuda.is_available())

False


In [6]:
print(logit_dataset)

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels', 'logit_values', 'logit_indices'],
        num_rows: 192701
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels', 'logit_values', 'logit_indices'],
        num_rows: 1949
    })
})


In [ ]:

teacher_model = AutoModelForCausalLM.from_pretrained(
    config.teacher_model_name,
    torch_dtype=torch.bfloat16,
)
teacher_model.resize_token_embeddings(new_num_tokens=config.student_vocab_size)
teacher_model.requires_grad_(False)
 

In [ ]:
teacher_eval_results = evaluate_model(teacher_model, dataset["test"], collator)

In [ ]:
print(teacher_eval_results["eval_loss"])
print(teacher_eval_results["perplexity"])

In [7]:

student_model = AutoModelForCausalLM.from_pretrained(
    config.student_model_name,
    torch_dtype=torch.bfloat16,
)

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


In [10]:
print("sampling 100 examples")
small_test = dataset["test"].select(range(100))
t0=time.time()
evaluate_model(student_model, small_test, collator)
print("elapsed:", time.time()-t0, "sec")

sampling 100 examples


KeyboardInterrupt: 